## Import Library

In [1]:
import pandas as pd
import re

## Load data

In [2]:
file_path = r'C:\Users\user\CAPSTONE-PROJEK---CODEY\Machine Learning\kategori soal\others.csv'

data = pd.read_csv(file_path)

data.head()

Instruction  \
0  Write a Python program to generate a Fibonacci...   
1  Identify the output of the following python co...   
2            Fix the following syntax in Python code   
3  Write a python program to successfully insert ...   
4                     Create a web server in Python.   

                                               Input  \
0                            Number of elements = 10   
1  arr = [1, 2, 4, 8, 16]\r\nnew_arr = [i * 2 for...   
2  string1 = "Programming"\r\nstring2 = "Fundamen...   
3      student_name = "John Doe"\r\nstudent_age = 20   
4                                         <no input>   

                                              Output   materi  question_id  
0  def generateFibonacci(numOfElements):\r\n sequ...  Lainnya            5  
1                                  [2, 4, 8, 16, 32]  Lainnya            6  
2  string1 = "Programming"\r\nstring2 = "Fundamen...  Lainnya            9  
3  import sqlite3\r\n\r\nconn = sqlite3.connect("...  Lainnya           12  
4  import http.server\r\nimport socketserver\r\n\...  Lainnya           20

In [3]:
def add_placeholder(code):
    """
    Gantikan kata setelah kata kunci dengan placeholder '___'.
    Hanya satu kali mengganti di seluruh kode setelah menemukan kecocokan pertama.
    """
    # Pola regex untuk mencari kata kunci dan mengganti kata setelahnya dengan placeholder
    patterns = [
         r'\b(import|open|read|write|close|with\s+open)\s+(\w+)\b'
    ]
    
    # Cari kecocokan pertama dalam seluruh kode
    for pattern in patterns:
        match = re.search(pattern, code)
        if match:
            # Ganti kata setelah kata kunci dengan placeholder '___' pada kecocokan pertama
            code = re.sub(pattern, r'\1 ___', code, count=1)  # Hanya mengganti kecocokan pertama
    
    return code


Menambah kolom Template untuk soal 

In [4]:
# Terapkan fungsi pada kolom 'Full Code'
data['Template'] = data['Output'].apply(add_placeholder)
print(data['Template'][3])

import ___

conn = sqlite3.connect("example.db")
c = conn.cursor()
c.execute("INSERT INTO students VALUES ('John Doe', 20)")
conn.commit()
conn.close()


Membuat fungsi untuk mengecek data soal memiliki placeholder

In [5]:
def check_placeholder_in_column(df, column_name, placeholder="___"):
    """
    Cek apakah kolom tertentu dalam DataFrame mengandung placeholder.
    
    Args:
        df (pd.DataFrame): DataFrame yang akan dicek.
        column_name (str): Nama kolom yang akan diperiksa.
        placeholder (str): Placeholder yang dicari (default: "___").
    
    Returns:
        pd.Series: Kolom boolean (True jika ada placeholder, False jika tidak).
    """
    return df[column_name].str.contains(placeholder, na=False)


In [6]:
data["has_placeholder"] = check_placeholder_in_column(data, "Template")

print(data["has_placeholder"].value_counts())

has_placeholder
False    1686
True      504
Name: count, dtype: int64


In [7]:
# Hapus baris dengan nilai False
data = data[data["has_placeholder"]]

print(data["has_placeholder"].value_counts())

# Hapus kolom 'has_placeholder' jika tidak diperlukan lagi
data = data.drop(columns=["has_placeholder"])


has_placeholder
True    504
Name: count, dtype: int64


In [8]:
def extract_keyword_variable(code):
    """
    Mengambil kata setelah kata kunci dan mengembalikannya.
    """
    # Pola regex untuk mencari kata kunci dan mengekstrak kata setelahnya
    pattern = r'\b(import|open|read|write|close|with\s+open)\s+(\w+)\b'
    
    match = re.search(pattern, code)
    
    # Jika ada kecocokan, ambil kata setelah kata kunci (hasil kedua dalam tuple)
    if match:
        return match[2]  # Kata setelah kata kunci
    
    return None

# Terapkan fungsi untuk membuat kolom baru
data['correct_answer'] = data['Output'].apply(extract_keyword_variable)



## Save Data

In [9]:
# Simpan dataset yang telah diproses
data.to_csv('others_with_placeholders.csv', index=False)

In [10]:
data.head()

Instruction  \
3   Write a python program to successfully insert ...   
4                      Create a web server in Python.   
16  Write a Python program that takes a sentence a...   
22  Write a Python program to solve a quadratic eq...   
23  Write a Python code to tokenize the given sent...   

                                            Input  \
3   student_name = "John Doe"\r\nstudent_age = 20   
4                                      <no input>   
16         String: "This, is a?! test* sentence."   
22                       a = 1\r\nb = -5\r\nc = 6   
23                      This is a sample sentence   

                                               Output   materi  question_id  \
3   import sqlite3\r\n\r\nconn = sqlite3.connect("...  Lainnya           12   
4   import http.server\r\nimport socketserver\r\n\...  Lainnya           20   
16  import string\r\n\r\ndef removePunctuations(se...  Lainnya           57   
22  import math\r\n\r\n# Solve the quadratic equat...  Lainnya           93   
23  import nltk\r\n\r\nsentence = "This is a sampl...  Lainnya           99   

                                             Template correct_answer  
3   import ___\r\n\r\nconn = sqlite3.connect("exam...        sqlite3  
4   import ___.server\r\nimport socketserver\r\n\r...           http  
16  import ___\r\n\r\ndef removePunctuations(sente...         string  
22  import ___\r\n\r\n# Solve the quadratic equati...           math  
23  import ___\r\n\r\nsentence = "This is a sample...           nltk

In [13]:
import json

# Membuat struktur JSON
json_data = {"quiz": []}

for _, row in data.iterrows():
    question_id = row['question_id']
    material = row['materi']
    instruction = row['Instruction']
    question = row['Template']
    attempts = '1'
    duration = '20'
    
    # Menggunakan regex untuk mencari placeholder '___' dan menggantinya dengan 'if'
    if '___' in question:
        correct_answer = row['correct_answer'] # Hanya menampilkan pengganti dari placeholder
    else:
        correct_answer = ""  # Jika tidak ada placeholder ___

    # Menambahkan ke struktur JSON
    question_entry = {
        "question_id": question_id,
        "material": material,
        "instruction": instruction,
        "question": question,
        "correct_answer": correct_answer,
        "attempts": attempts,
        "duration": duration
    }
    json_data["quiz"].append(question_entry)

# Menyimpan ke file JSON
output_json_path = 'questions_others.json'
with open(output_json_path, 'w') as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"File JSON berhasil disimpan: {output_json_path}")


File JSON berhasil disimpan: questions_others.json


In [12]:
data.head()

Instruction  \
3   Write a python program to successfully insert ...   
4                      Create a web server in Python.   
16  Write a Python program that takes a sentence a...   
22  Write a Python program to solve a quadratic eq...   
23  Write a Python code to tokenize the given sent...   

                                            Input  \
3   student_name = "John Doe"\r\nstudent_age = 20   
4                                      <no input>   
16         String: "This, is a?! test* sentence."   
22                       a = 1\r\nb = -5\r\nc = 6   
23                      This is a sample sentence   

                                               Output   materi  question_id  \
3   import sqlite3\r\n\r\nconn = sqlite3.connect("...  Lainnya           12   
4   import http.server\r\nimport socketserver\r\n\...  Lainnya           20   
16  import string\r\n\r\ndef removePunctuations(se...  Lainnya           57   
22  import math\r\n\r\n# Solve the quadratic equat...  Lainnya           93   
23  import nltk\r\n\r\nsentence = "This is a sampl...  Lainnya           99   

                                             Template correct_answer  
3   import ___\r\n\r\nconn = sqlite3.connect("exam...        sqlite3  
4   import ___.server\r\nimport socketserver\r\n\r...           http  
16  import ___\r\n\r\ndef removePunctuations(sente...         string  
22  import ___\r\n\r\n# Solve the quadratic equati...           math  
23  import ___\r\n\r\nsentence = "This is a sample...           nltk